In [1]:
import os

import cv2
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm


In [2]:
# check whether we are in the right directory
os.getcwd() + "/database/" + os.listdir("database")[0]

'/Users/aditya/Desktop/desktop/robot-perception-hw/HW2/database/image18558.png'

In [3]:
"""
Extract featues using SIFT, and store them in a list.
"""

def extract_features(image, extractor):

    # compute descriptors
    keypoints, descriptors = extractor.detectAndCompute(image, None)

    return keypoints, descriptors


In [4]:
keypoint_list, descriptor_list = [], []
extractor = cv2.SIFT_create()
data_path = "database"

# compute descriptor for all images in the database
for img_path in tqdm(os.listdir(data_path), desc="Feature extraction: "):

    # access curent image and convert it to grayscale
    curr_img_path = os.getcwd() + "/" + data_path + "/" + img_path
    curr_img = cv2.imread(curr_img_path, cv2.IMREAD_GRAYSCALE)

    # compute descriptors
    keypoints, descriptors = extract_features(curr_img, extractor)

    # store keypoints and descriptors
    keypoint_list.append(keypoints)
    descriptor_list.append(descriptors)

Feature extraction: 100%|██████████| 28600/28600 [01:50<00:00, 258.55it/s]


In [5]:
# vertically stack sift features to be fed into K means
descriptor_list = np.vstack(tuple(descriptor_list))

In [6]:
descriptor_list.shape

(6170916, 128)

In [7]:
# Compute 300 clusters of descriptors. Using minibatch k means because 
# it converges faster( according to scipy documentation)

# define the k means model with parameters
model = MiniBatchKMeans(n_clusters=300, init="k-means++", verbose=1)

# compute clusters
model.fit(descriptor_list)

Init 1/3 with method k-means++
Inertia for init 1/3: 386619232.0
Init 2/3 with method k-means++
Inertia for init 2/3: 386581984.0
Init 3/3 with method k-means++
Inertia for init 3/3: 387507232.0
[MiniBatchKMeans] Reassigning 31 cluster centers.
Minibatch step 1/602628: mean batch inertia: 125580.0390625
Minibatch step 2/602628: mean batch inertia: 99982.28125, ewa inertia: 99982.28125
Minibatch step 3/602628: mean batch inertia: 95181.8671875, ewa inertia: 99980.6880916412
Minibatch step 4/602628: mean batch inertia: 93787.71875, ewa inertia: 99978.63277292412
Minibatch step 5/602628: mean batch inertia: 92871.078125, ewa inertia: 99976.27392221864
Minibatch step 6/602628: mean batch inertia: 92777.046875, ewa inertia: 99973.88464733571
Minibatch step 7/602628: mean batch inertia: 91340.7890625, ewa inertia: 99971.01950107662
Minibatch step 8/602628: mean batch inertia: 91503.140625, ewa inertia: 99968.2091868335
Minibatch step 9/602628: mean batch inertia: 90628.0078125, ewa inertia: 

KeyboardInterrupt: 

In [10]:
# check whether we got 300 clusters
model.cluster_centers_.shape

(300, 128)

In [ ]:
def build_histogram(descriptor_list, model):
    histogram = np.zeros(len(model.cluster_centers_))
    cluster_result =  model.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return histogram

In [22]:
# find which cluster a feature belongs to.
clusters_indexes = model.predict(descriptor_list)